This file can be run to upload images and their tags (object detection bounding box coordinates) to our object decetion model <br/>
It uses the tagged-images.json file which contains bounding box coordinates for objects in multiple images and the images in the /images/more_training_images subfolder

In [4]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import time
import json

import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
training_endpoint = os.getenv('CUSTOM_IMAGE_TRAIN_ENDPOINT')
training_key = os.getenv('CUSTOM_IMAGE_TRAIN_KEY')
project_id = '97462dc8-2998-4044-951f-ed14d6ad89d9' #you could add this to .env -- and best practice probably says to

In [5]:
# Authenticate a client for the training API
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
training_client = CustomVisionTrainingClient(training_endpoint, credentials)

In [6]:
# Get the Custom Vision project
custom_vision_project = training_client.get_project(project_id)

In [7]:
def Upload_Images(folder):
    '''
    This function takes a filepath in and goes through all of the images in that file path, as well as collecting the cooresponding object tags given in the 04a. tagged-images.json file,
    and uploads them to our object detection project. 
    '''
    print("Uploading images...")

    # Get the tags defined in the project
    tags = training_client.get_tags(custom_vision_project.id)

    # Create a list of images with tagged regions
    tagged_images_with_regions = []

    # Get the images and tagged regions from the JSON file
    with open('04a. tagged-images.json', 'r') as json_file:
        tagged_images = json.load(json_file)
        for image in tagged_images['files']:
            # Get the filename
            file = image['filename']
            # Get the tagged regions
            regions = []
            for tag in image['tags']:
                tag_name = tag['tag']
                # Look up the tag ID for this tag name
                tag_id = next(t for t in tags if t.name == tag_name).id
                # Add a region for this tag using the coordinates and dimensions in the JSON
                regions.append(Region(tag_id=tag_id, left=tag['left'],top=tag['top'],width=tag['width'],height=tag['height']))
            # Add the image and its regions to the list
            with open(os.path.join(folder,file), mode="rb") as image_data:
                tagged_images_with_regions.append(ImageFileCreateEntry(name=file, contents=image_data.read(), regions=regions))

    # Upload the list of images as a batch
    upload_result = training_client.create_images_from_files(custom_vision_project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
    # Check for failure
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
    else:
        print("Images uploaded.")

In [8]:
# Upload and tag images
Upload_Images('images/Object Detection/more_training_images')

Uploading images...
Images uploaded.
